In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

path = "ImagePro"
files=os.listdir(path)

files.sort()

print(files)

image_array=[]
label_array=[]
for i in range(len(files)):
    #list of image in each folder
    sub_file= os.listdir(path+"/"+files[i])
    #print(len(sub_file))
    
    #loop through each subfile
    for j in tqdm(range(len(sub_file))):
        #path of each image
        # Example : Imagepro/A/image_name1.jpg
        file_path=path+"/"+files[i]+"/"+sub_file[j]
        #read each image
        
        image=cv2.imread(file_path)
        
        #resize image by 96x96
        image = cv2.resize(image,(96,96))
        #convert BGR image to RGB image
        
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        #add this image to image_array
        
        image_array.append(image)
        
        #add label to label array
        # i is number form 0 to len(files)-1
        #so we can use it as label
        label_array.append(i)
        

image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,Y_test = train_test_split(image_array,label_array,test_size=0.15)


del image_array,label_array
#X_train Will have 85% of images
#X_test will have 15%of images
import gc
gc.collect()
#Create a model

from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

model= Sequential()
# add pretrained models to sequentail model
# i will use EfficientNetB0 pretrained model 
pretrained_model= tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)
model.add(pretrained_model)

#add Pooling to model

model.add(layers.GlobalAveragePooling2D())

# add dropout to model
#we add droupout to increase accracy by reduce overfitting
model.add(layers.Dropout(0.3))

#finally we will add dense layer to output
model.add(layers.Dense(1))

# For some tensorflow version we required to build model
model.build(input_shape=(None,96,96,3))

#to see model summary
model.summary()


#save and run 

model.compile(optimizer="adam", loss = "mae", metrics =["mae"])

# create a checkpoint to save best acuracy model

ckp_path="trained_model/model"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,monitor="val_mae", save_best_only=True, save_weights_only=True)

#monitor: monitor validation mae loss to save model
#mode ; use to save model  when it is minimum or maximum
# It has 3 option: "min","max", "auto"
# for us you can select either "min" or "auto"

#when val_mae reduce model will be saved
# save_best_only: False -> It will save all model
#save_weight_olny: Save only weight.

#create learning rate reducer to reduce lr when accuracy does not import

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                         mode="auto", cooldown = 0,
                                        patience = 5,
                                        verbose =1, min_lr=1e-6)

# factor: when it is reduce next lr will be 0.9 times of current
# next lr = 0.9*current lr

# patience= X
# reduce lr after x epoch when accuracy does not imporve
# verbose: show it after ever epoch

#min_lr: minimum learning rate

# Start training model

Epoch = 100
Batch_Size= 32

# select batch size accordingly to your Graph card 
# 

history = model.fit(
                    X_train,
                    Y_train,
                    validation_data = (X_test, Y_test),
                    batch_size = Batch_Size,
                    epochs = Epoch,
                    callbacks = [model_checkpoint,reduce_lr])

model.load_weights(ckp_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

with open("model.tflite","wb") as f:
    f.write(tflite_model)

    
prediction_val=model.predict(X_test, batch_size = 32)

print(prediction_val[:10])
print(Y_test[:10])



        
        
        
        
        
        

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


100%|██████████| 469/469 [00:01<00:00, 235.42it/s]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/100
303/303 [==============================] - 373s 1s/step - loss: 3.1498 - mae: 3.1498 - val_loss

Epoch 89/100
303/303 [==============================] - 324s 1s/step - loss: 0.2183 - mae: 0.2183 - val_loss: 0.1354 - val_mae: 0.1354 - lr: 2.5419e-04
Epoch 90/100
303/303 [==============================] - 327s 1s/step - loss: 0.2124 - mae: 0.2124 - val_loss: 0.1103 - val_mae: 0.1103 - lr: 2.5419e-04
Epoch 91/100
303/303 [==============================] - 336s 1s/step - loss: 0.2096 - mae: 0.2096 - val_loss: 0.2229 - val_mae: 0.2229 - lr: 2.5419e-04
Epoch 92/100
303/303 [==============================] - 324s 1s/step - loss: 0.2084 - mae: 0.2084 - val_loss: 0.2084 - val_mae: 0.2084 - lr: 2.5419e-04
Epoch 93/100
303/303 [==============================] - 343s 1s/step - loss: 0.2108 - mae: 0.2108 - val_loss: 0.0846 - val_mae: 0.0846 - lr: 2.5419e-04
Epoch 94/100
303/303 [==============================] - 342s 1s/step - loss: 0.2112 - mae: 0.2112 - val_loss: 0.0866 - val_mae: 0.0866 - lr: 2.5419e-04
Epoch 95/100
303/303 [==============================] - 384s 1s/step - loss: 0.2114 - ma

INFO:tensorflow:Assets written to: C:\Users\Anurag\AppData\Local\Temp\tmpujeefpq4\assets


INFO:tensorflow:Assets written to: C:\Users\Anurag\AppData\Local\Temp\tmpujeefpq4\assets


54/54 [==============================] - 31s 360ms/step
[[18.072456 ]
 [17.027466 ]
 [ 3.9789872]
 [ 7.03244  ]
 [18.055027 ]
 [18.965885 ]
 [15.044263 ]
 [20.151802 ]
 [ 5.061651 ]
 [ 3.0553079]]
[18. 17.  4.  7. 18. 19. 15. 20.  5.  3.]
